In [1]:
path_model = 'C:/Users/sOrOush/SoroushProjects/02_Linearity_Project/02_Models/vdvae'

import sys
sys.path.append(path_model)
# Check if the directory has been added
print("Current sys.path:", sys.path)

# Attempt to import the module
try:
    from model_utils import *
    print("Module model_utils successfully imported.")
except ImportError as e:
    print(f"Error importing the module: {e}")


Current sys.path: ['C:\\Users\\sOrOush\\SoroushProjects\\14_CLIP_Ozcelic', 'C:\\Program Files\\JetBrains\\PyCharm 2024.3.2\\plugins\\python-ce\\helpers\\pydev', 'C:\\Program Files\\JetBrains\\PyCharm 2024.3.2\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\python312.zip', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\DLLs', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question', '', 'C:\\Users\\sOrOush\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\win32', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\setupto

In [2]:

import torch
import numpy as np
#from mpi4py import MPI
import socket
import argparse
import os
import json
import subprocess
from hps import Hyperparams, parse_args_and_update_hparams, add_vae_arguments
from utils import (logger,
                   local_mpi_rank,
                   mpi_size,
                   maybe_download,
                   mpi_rank)
from data import mkdir_p
from contextlib import contextmanager
import torch.distributed as dist
#from apex.optimizers import FusedAdam as AdamW
from vae import VAE
from torch.nn.parallel.distributed import DistributedDataParallel
from train_helpers import restore_params
from image_utils import *
from model_utils import *
from torch.utils.data import DataLoader, Dataset
from PIL import Image
import torchvision.transforms as T
import pickle

import argparse

In [3]:
# Check if the script is running in a Jupyter Notebook
if 'ipykernel' in sys.modules:
    # Default values for Jupyter Notebook
    sub = 1  # You can specify the desired default value here
    batch_size = 30  # Default value for batch size
else:
    # Argument parser for command-line execution
    parser = argparse.ArgumentParser(description='Argument Parser')
    parser.add_argument("-sub", "--sub", help="Subject Number", default=7)
    parser.add_argument("-bs", "--bs", help="Batch Size", default=30)
    args = parser.parse_args()

    # Set the subject number and ensure it's valid
    sub = int(args.sub)
    assert sub in [1, 2, 5, 7]  # Validate allowed subject numbers

    # Set the batch size
    batch_size = int(args.bs)

# Print status messages
print('Libraries imported')
print(f'Using Subject Number: {sub}')
print(f'Using Batch Size: {batch_size}')


Libraries imported
Using Subject Number: 1
Using Batch Size: 30


In [4]:
# Check if the path has been added
print("Current sys.path:", sys.path)


Current sys.path: ['C:\\Users\\sOrOush\\SoroushProjects\\14_CLIP_Ozcelic', 'C:\\Program Files\\JetBrains\\PyCharm 2024.3.2\\plugins\\python-ce\\helpers\\pydev', 'C:\\Program Files\\JetBrains\\PyCharm 2024.3.2\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\python312.zip', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\DLLs', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question', '', 'C:\\Users\\sOrOush\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\win32', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\setupto

In [5]:
import os

# Check if the file exists
model_path = r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-model-ema.th'
if os.path.exists(model_path):
    print(f"The file exists: {model_path}")
else:
    print(f"File not found: {model_path}")


The file exists: C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-model-ema.th


In [6]:
import numpy as np
import torch
from PIL import Image
from torch.utils.data import Dataset
import torchvision.transforms as T

# Define the configuration dictionary
H = {
    'image_size': 64, 'image_channels': 3, 'seed': 0, 'port': 29500, 'save_dir': './saved_models/test',
    'data_root': './', 'desc': 'test', 'hparam_sets': 'imagenet64',
    'restore_path': r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-model.th',
    'restore_ema_path': r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-model-ema.th',
    'restore_log_path': r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-log.jsonl',
    'restore_optimizer_path': r'C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-opt.th',
    'dataset': 'imagenet64', 'ema_rate': 0.999, 'enc_blocks': '64x11,64d2,32x20,32d2,16x9,16d2,8x8,8d2,4x7,4d4,1x5',
    'dec_blocks': '1x2,4m1,4x3,8m4,8x7,16m8,16x15,32m16,32x31,64m32,64x12', 'zdim': 16, 'width': 512,
    'custom_width_str': '', 'bottleneck_multiple': 0.25, 'no_bias_above': 64, 'scale_encblock': False,
    'test_eval': True, 'warmup_iters': 100, 'num_mixtures': 10, 'grad_clip': 220.0, 'skip_threshold': 380.0,
    'lr': 0.00015, 'lr_prior': 0.00015, 'wd': 0.01, 'wd_prior': 0.0, 'num_epochs': 10000, 'n_batch': 4,
    'adam_beta1': 0.9, 'adam_beta2': 0.9, 'temperature': 1.0, 'iters_per_ckpt': 25000, 'iters_per_print': 1000,
    'iters_per_save': 10000, 'iters_per_images': 10000, 'epochs_per_eval': 1, 'epochs_per_probe': None,
    'epochs_per_eval_save': 1, 'num_images_visualize': 8, 'num_variables_visualize': 6, 'num_temperatures_visualize': 3,
    'mpi_size': 1, 'local_rank': 0, 'rank': 0, 'logdir': './saved_models/test/log'
}

# Dot notation for dictionary
class dotdict(dict):
    """dot.notation access to dictionary attributes"""
    __getattr__ = dict.get
    __setattr__ = dict.__setitem__
    __delattr__ = dict.__delitem__

H = dotdict(H)

# Assuming `set_up_data` and `load_vaes` functions are defined elsewhere
H, preprocess_fn = set_up_data(H)

print('Model is Loading')
ema_vae = load_vaes(H)

# Dataset class for external images
class batch_generator_external_images(Dataset):
    def __init__(self, data_path):
        self.data_path = data_path
        self.im = np.load(data_path).astype(np.uint8)

    def __getitem__(self, idx):
        img = Image.fromarray(self.im[idx])
        img = T.functional.resize(img, (64, 64))
        img = torch.tensor(np.array(img)).float()
        return img

    def __len__(self):
        return len(self.im)


Model is Loading
Restoring ema vae from C:\Users\sOrOush\SoroushProjects\02_Linearity_Project\02_Models\vdvae\model\imagenet64-iter-1600000-model-ema.th


C:\Users/sOrOush/SoroushProjects/02_Linearity_Project/02_Models/vdvae\train_helpers.py:126: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  state_dict = torch.load(distributed

In [8]:
import sys

# Append the desired path to sys.path
sub = 1  # Replace with the appropriate subject number if needed
sys.path.append(r"C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata")

print("Updated sys.path:", sys.path)


Updated sys.path: ['C:\\Users\\sOrOush\\SoroushProjects\\14_CLIP_Ozcelic', 'C:\\Program Files\\JetBrains\\PyCharm 2024.3.2\\plugins\\python-ce\\helpers\\pydev', 'C:\\Program Files\\JetBrains\\PyCharm 2024.3.2\\plugins\\python\\helpers-pro\\jupyter_debug', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\python312.zip', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\DLLs', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question', '', 'C:\\Users\\sOrOush\\AppData\\Roaming\\Python\\Python312\\site-packages', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\win32', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\win32\\lib', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\Pythonwin', 'C:\\Users\\sOrOush\\.conda\\envs\\linearity_question\\Lib\\site-packages\\setupto

In [9]:
import os

# Display the current working directory
current_directory = os.getcwd()
print("Current working directory:", current_directory)


Current working directory: C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\scripts


In [13]:
import os

# Define the image path
sub = 1  # Replace with the appropriate subject number
image_path = r"C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata\subj{sub:02d}\nsd_test_stim_sub{sub}.npy"

# Check if the file exists
if os.path.exists(image_path):
    print(f"The file exists: {image_path}")
else:
    print(f"File not found: {image_path}")


File not found: C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata\subj{sub:02d}\nsd_test_stim_sub{sub}.npy


In [20]:
import os

# Define the image path
sub = 1  # Replace with the appropriate subject number
image_path = rf"C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata\subj{sub:02d}\nsd_train_stim_sub{sub}.npy"

# Check if the file exists
if os.path.exists(image_path):
    print(f"The file exists: {image_path}")
else:
    print(f"File not found: {image_path}")


The file exists: C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata\subj01\nsd_train_stim_sub1.npy


In [16]:
import torch
import gc

# Clear the CUDA cache
torch.cuda.empty_cache()

# Force garbage collection
gc.collect()

# If you have variables holding GPU tensors, delete them
# For example:
# del model, optimizer, data_loader  # Replace with your variables
# gc.collect()  # Re-run garbage collection after deletion

# Check if VRAM is cleared
print("GPU memory usage after cleanup:")
print(f"Allocated: {torch.cuda.memory_allocated()} bytes")
print(f"Cached: {torch.cuda.memory_reserved()} bytes")


GPU memory usage after cleanup:
Allocated: 500083200 bytes
Cached: 530579456 bytes


In [17]:
import torch

def get_gpu_properties():
    if not torch.cuda.is_available():
        print("No GPU available. Please ensure CUDA is installed and a GPU is accessible.")
        return None

    gpu_properties = torch.cuda.get_device_properties(0)
    print(f"GPU Name: {gpu_properties.name}")
    print(f"Total VRAM: {gpu_properties.total_memory / 1e9:.2f} GB")
    print(f"Compute Capability: {gpu_properties.major}.{gpu_properties.minor}")
    print(f"Device Memory Allocated: {torch.cuda.memory_allocated(0) / 1e9:.2f} GB")
    print(f"Device Memory Cached: {torch.cuda.memory_reserved(0) / 1e9:.2f} GB")

    return gpu_properties

def suggest_batch_size(vram_gb, model_size_gb=4, buffer_size_gb=1):
    """
    Suggests a batch size based on available VRAM.

    Parameters:
        vram_gb (float): Total GPU memory in GB.
        model_size_gb (float): Approximate size of the model in memory in GB.
        buffer_size_gb (float): Reserved space for overheads (activations, caching).

    Returns:
        int: Suggested batch size.
    """
    usable_vram = vram_gb - (model_size_gb + buffer_size_gb)
    if usable_vram <= 0:
        print("Insufficient GPU memory for the model and overheads.")
        return 1

    # Assume each data sample requires ~0.01 GB (10 MB) as a heuristic
    batch_size = int(usable_vram / 0.01)
    return batch_size

# Get GPU properties
gpu_props = get_gpu_properties()

if gpu_props:
    # Suggest batch size
    total_vram_gb = gpu_props.total_memory / 1e9  # Convert bytes to GB
    recommended_batch_size = suggest_batch_size(total_vram_gb)

    print(f"Recommended Batch Size: {recommended_batch_size}")


GPU Name: NVIDIA GeForce RTX 4080 Laptop GPU
Total VRAM: 12.88 GB
Compute Capability: 8.9
Device Memory Allocated: 0.50 GB
Device Memory Cached: 0.53 GB
Recommended Batch Size: 787


In [18]:
import torch
import numpy as np
import os
from torch.utils.data import DataLoader

def process_latents(trainloader, testloader, num_latents, batch_size, ema_vae, preprocess_fn):
    train_latents = []
    test_latents = []
    ref_latent = None

    # Process training data
    for i, x in enumerate(trainloader):
        data_input, target = preprocess_fn(x)
        with torch.no_grad():
            print(f"Processing training batch {i*batch_size}")
            activations = ema_vae.encoder.forward(data_input)
            px_z, stats = ema_vae.decoder.forward(activations, get_latents=True)
            batch_latent = []
            for j in range(num_latents):
                batch_latent.append(stats[j]['z'].cpu().numpy().reshape(len(data_input), -1))
            train_latents.append(np.hstack(batch_latent))

    # Process test data
    for i, x in enumerate(testloader):
        data_input, _ = preprocess_fn(x)
        with torch.no_grad():
            print(f"Processing test batch {i*batch_size}")
            activations = ema_vae.encoder.forward(data_input)
            px_z, stats = ema_vae.decoder.forward(activations, get_latents=True)
            if i == 0:
                ref_latent = stats
            batch_latent = []
            for j in range(num_latents):
                batch_latent.append(stats[j]['z'].cpu().numpy().reshape(len(data_input), -1))
            test_latents.append(np.hstack(batch_latent))

    return (np.concatenate(train_latents),
            np.concatenate(test_latents),
            ref_latent)


In [21]:
def main():
    sub = 1  # Replace with desired subject number
    batch_size = 30  # Updated batch size
    latent_sizes = [31]  # Updated latent sizes list

    # Load data
    train_image_path = r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata\subj{:02d}\nsd_train_stim_sub{}.npy'.format(sub, sub)
    test_image_path = r'C:\Users\sOrOush\SoroushProjects\01_Soroush_and_Shakiba\NSD_High_Dimensional_Data\11_Marco_And_Soroush\Data\processed_nsddata\subj{:02d}\nsd_test_stim_sub{}.npy'.format(sub, sub)

    train_images = batch_generator_external_images(data_path=train_image_path)
    test_images = batch_generator_external_images(data_path=test_image_path)

    trainloader = DataLoader(train_images, batch_size=batch_size, shuffle=False)
    testloader = DataLoader(test_images, batch_size=batch_size, shuffle=False)

    # Process each latent size
    for num_latents in latent_sizes:
        print(f"\nProcessing {num_latents} latent spaces...")

        # Create directory
        dir_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results\extracted_fetures_{}_Layered\subj{:02d}".format(num_latents, sub)
        os.makedirs(dir_path, exist_ok=True)

        # Process and save latents
        train_latents, test_latents, ref_latent = process_latents(
            trainloader, testloader, num_latents, batch_size, ema_vae, preprocess_fn
        )

        # Save results
        np.savez(f"{dir_path}\\nsd_vdvae_features_{num_latents}l.npz",
                 train_latents=train_latents,
                 test_latents=test_latents)
        np.savez(f"{dir_path}/ref_latents_{num_latents}l.npz",
                 ref_latent=ref_latent)

        print(f"Saved latents for {num_latents} layers")

if __name__ == "__main__":
    main()



Processing 31 latent spaces...
Processing training batch 0
Processing training batch 30
Processing training batch 60
Processing training batch 90
Processing training batch 120
Processing training batch 150
Processing training batch 180
Processing training batch 210
Processing training batch 240
Processing training batch 270
Processing training batch 300
Processing training batch 330
Processing training batch 360
Processing training batch 390
Processing training batch 420
Processing training batch 450
Processing training batch 480
Processing training batch 510
Processing training batch 540
Processing training batch 570
Processing training batch 600
Processing training batch 630
Processing training batch 660
Processing training batch 690
Processing training batch 720
Processing training batch 750
Processing training batch 780
Processing training batch 810
Processing training batch 840
Processing training batch 870
Processing training batch 900
Processing training batch 930
Processing tr

In [30]:
base_path = r"C:\Users\sOrOush\SoroushProjects\14_CLIP_Ozcelic\results"

layer_path = f"{base_path}\\extracted_fetures_{31}_Layered\\subj{1:02d}\\nsd_vdvae_features_{31}l.npz"
nsd_features = np.load(layer_path)

In [32]:
train_latents = nsd_features['train_latents'].astype(np.float32)
test_latents = nsd_features['test_latents'].astype(np.float32)

In [34]:
train_latents.shape

(8859, 91168)